In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import vgg16
from dataclasses import dataclass
from pathlib import Path
from secondClassifier.config.configuration import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from secondClassifier.utils import read_yaml, create_directory

In [2]:
os.chdir('../')

### Defining Data class

In [3]:
@dataclass(frozen=True)
class PrepareBasemodelConfig:
    root_dir:Path
    base_model_path: Path
    updated_model_path: Path
    include_top: bool
    image_size: list
    learning_rate: float
    weights: str
    classes: int

In [4]:
class ConfigurationManager:
    def __init__(self,
                  config_filepath= CONFIG_FILE_PATH,
                  params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directory(self.config.artifacts_root)

    def get_base_model_config(self)->PrepareBasemodelConfig:
        basemodel_details= self.config.prepare_base_model
        ##Create root directory for prepare basemodel

        
        base_model_config= PrepareBasemodelConfig(
            root_dir=Path(basemodel_details.root_dir),
            base_model_path=Path(basemodel_details.base_model_path),
            updated_model_path=Path(basemodel_details.updated_model_path),
            include_top=self.params.INCLUDE_TOP,
            image_size=self.params.IMAGE_SIZE,
            learning_rate=self.params.LEARNING_RATE,
            weights=self.params.WEIGHTS,
            classes=self.params.CLASSES
        )
        return base_model_config

### Preparebase model class and updating the model

In [48]:
class PrepareBaseModel:
    def __init__(self, config= PrepareBasemodelConfig):
        self.config= config
    
    def get_base_model(self):
        ##Get VGG16 model
        #print(self.config.include_top)
        self.base_model= vgg16.VGG16(
            include_top=self.config.include_top,
            weights=self.config.weights,
            classes= self.config.classes,
            input_shape= self.config.image_size
        )
        base_model_path= self.config.base_model_path
        self.save_model(model=self.base_model, path=base_model_path)
        #return base_model
    
    def update_base_model(self):
        """Get the base model and create a new model out of it
        """
        ##Get the base model and check to train or un-train
        for layer_num in range (len(self.base_model.layers)):
            self.base_model.layers[layer_num].trainable=False
        ##Create ANN
        flatten= Flatten()(self.base_model.output)
        output_layer= Dense(units=5, activation='softmax')(flatten)

        ##Create new model
        update_model= tf.keras.models.Model(
            inputs= self.base_model.input,
            outputs= output_layer
        )
        ##Compile the model
        update_model.compile(
            loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.learning_rate),
            metrics= ["accuracy"]
        )
        #update_model.summary()
        #update_model.save(self.config.updated_model_path)
        updated_model_path= self.config.updated_model_path
        self.save_model(model=update_model, path=updated_model_path)
       #return update_model
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


### Practice

In [49]:
config= ConfigurationManager()
base_model_config= config.get_base_model_config()
preparemodel= PrepareBaseModel(base_model_config)
preparemodel.get_base_model()
preparemodel.update_base_model()

In [45]:
updated_model.get_config()

{'name': 'model_14',
 'trainable': True,
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 224, 224, 3),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_19'},
   'name': 'input_19',
   'inbound_nodes': []},
  {'class_name': 'Conv2D',
   'config': {'name': 'block1_conv1',
    'trainable': False,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'same',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'block1_conv1',
   'inbound_nodes': [[['input_19', 0, 0, {}]]]},
  

In [42]:
updated_model.save('artifacts/prepare_base_model/updated_model_2.h5')

TypeError: get_config() missing 1 required positional argument: 'self'